本脚本目的为了解析pdf，将文字及图片提取出来

# 使用langchian自带工具提取

看看是否能提取双栏布局、图片信息

In [1]:
from langchain.document_loaders import PyPDFLoader

# 定义文本加载器，指定读取的文件路径
loader = PyPDFLoader("data.pdf")
documents=loader.load()

In [8]:
from IPython.display import Markdown

display(Markdown(documents[27].page_content))

装载货物
28
注意！
■ 在将座椅靠背向下折叠之前，移除头枕。否则，头枕可能会受到
损坏。
■ 折叠座椅靠背前，确保座椅上没有物体且两侧安全带绕开座椅靠
背，否则座椅可能会受到损坏。
使用手套箱密码保护
您可以使用密码锁功能锁住手套箱。当您的车辆停在维修站或者使用
代客泊车服务时，该功能可以避免手套箱被打开。
启用/停用手套箱密码保护功能
在中央显示屏中点击
 -座舱体验-快捷设置，进入快捷设置界面。
01 点击启用/停用手套箱密码保护功能。
■ 锁止：点击快捷设置功能界面中的手套箱密码锁按键，在弹出界
面中设定密码，锁止手套箱。
■ 解锁：点击快捷设置功能界面中的手套箱密码锁按键或按下手套
箱开关，在弹出界面中输入密码解锁手套箱。
说明！
□ 通过密码解锁手套箱后，手套箱密码保护功能自动关闭，如需使
用需重新开启。
□ 解锁手套箱时，若密码输入错误超过五次，手套箱密码保护功能
将进入保护状态，2 分钟后才可再次输入密码。此时，您也可以
通过手机验证码进行解锁。
后备厢载物
后备厢具备有用于固定行李件的挂钩。


观察文档发现：文档按3级层次编写，如(泊车-泊车辅助摄像头局限性-具体内容)，文字内容、图片在第三级别上，所以我们解析最好按照这个格式处理

由于langchian自带解析器无法进行自定义解析，因此自定义pdf解析器

## 自定义解析器

In [1]:
import io
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pymupdf # imports the pymupdf library

pages = pymupdf.open("data.pdf") # open a document

img_idx=0
doc_idx=0
save_flag=False
lines_text=''
save_page_idx=''
save_page_type=''

for _,page in enumerate(pages):
    page_text = page.get_text("dict")
    blocks = page_text["blocks"]  # the list of block dictionaries
    
    if len(blocks)<=2:
        continue
    # print(blocks)
    page_type=blocks[0]['lines'][0]['spans'][0]['text']
    page_idx=str(blocks[1]['lines'][0]['spans'][0]['text'])

    for idx in range(2,len(blocks)):
        block=blocks[idx]
        # print(block)
        if block["type"] == 0:
            lines=block['lines']
            for line in lines:
                spans=line['spans']
                spans_text=''
                for span in spans:
                    if span['size']>9.9 and span['font']=='FZLTZCHK--GBK1-0':
                        if save_flag==True and len(lines_text)>0:
                            with open(f'docs/doc{save_page_idx}-{save_page_type}-{doc_idx}.md','w') as fw:
                                fw.write(lines_text)
                            lines_text=''
                            if spans_text.startswith('#'):
                                spans_text+=span['text']
                            else:
                                spans_text="# "+span['text']
                            doc_idx+=1
                            save_flag=False
                        else:
                            if spans_text.startswith('#'):
                                spans_text+=span['text']
                            else:
                                spans_text="# "+span['text']
                    else:
                        if span['size']>9.9 and span['font']=='FZLTHJW--GB1-0':
                            if spans_text.startswith('##'):
                                spans_text+=span['text']
                            else:
                                spans_text="## "+span['text']
                        else:
                            save_flag=True
                            save_page_idx=page_idx
                            save_page_type=page_type
                            spans_text+=span['text']

                lines_text+=spans_text+'\n'
        if block["type"] == 1:
            img_bytes=block['image']
            width=block['width']
            height=block['height']
            colorspace=block['colorspace']
            image_data=io.BytesIO(img_bytes)
            img=Image.open(image_data)
            
            if width >500 and height>500:
                image_name=f'img{page_idx}-{img_idx}'+'.png'
                img.save(os.path.join('images',image_name),format='png')
                lines_text+='|'+image_name+'|'
                img_idx+=1

with open(f'docs/doc{save_page_idx}-{doc_idx}.md','w') as fw:
    fw.write(lines_text)

加载效果展示：

![1](doc/2024-12-17_114000.png)

上图能应对双栏排版，但是无法提取图片

![2](doc/2024-12-17_114753.png)

上图不能识别同一标题下的跨页问题

以上代码将pdf拆分为"标题-内容'的格式，并将每个标题内容保存，命名为“页号-文档类型-内容编号”，如下图所示

![alt text](doc/2024-12-17_164623.png)

注意：文档内容包括了图片